In [5]:
!pip install langchain_community langchain langchainhub tavily-python langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 17.3 MB/s eta 0:00:0000:01
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.4.0
    Uninstalling google-ai-generativelanguage-0.4.0:
      Successfully uninstalled google-ai-generativelanguage-0.4.0
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.4.1
    Uninstalling google-generativeai-0.4.1:
      Successfully uninstalled google-generativeai-0.4.1


In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_google_genai import ChatGoogleGenerativeAI

## Api Keys

In [3]:
# Loading Gemini_API_KEY and TAVILY_API_KEY from environment variables
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
gemini_api_key = user_secrets.get_secret("GEMINI")
TAVILY_API_KEY = user_secrets.get_secret("TAVILY_API_KEY")

You can get TAVILY_API_KEY here: https://app.tavily.com/

In [4]:

def is_key_valid(key):
    return key is not None and key.strip() != ""

user_secrets = UserSecretsClient()

GEMINI = user_secrets.get_secret("GEMINI")
if is_key_valid(GEMINI):
    print("GEMINI API key loaded successfully.")
else:
    print("Failed to load GEMINI API key or the key is invalid.")

TAVILY_API_KEY = user_secrets.get_secret("TAVILY_API_KEY")
if is_key_valid(TAVILY_API_KEY):
    print("Tavily API key loaded successfully.")
else:
    print("Failed to load Tavily API key or the key is invalid.")

GEMINI API key loaded successfully.
Tavily API key loaded successfully.


## Agent and Prompt Template

In [8]:
instructions = """You are an experienced researcher who always finds high-quality and relevant information on the Internet."""

base_prompt = hub.pull("langchain-ai/openai-functions-template")

prompt = base_prompt.partial(instructions=instructions)

/opt/conda/lib/python3.10/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    api_key=GEMINI
)

# Initialize a custom search tool named TavilySearchResults
api_wrapper = TavilySearchAPIWrapper(tavily_api_key=TAVILY_API_KEY)
tavily_tool = TavilySearchResults(api_wrapper=api_wrapper)

# Aggregate the tools into a list for easy access
tools = [tavily_tool]

# Create an AI agent with the specified LLM and tools, and the customized prompt
agent = create_openai_functions_agent(llm, 
                                      tools, 
                                      prompt)

# Set up an executor for the agent, specifying the agent, tools, and enabling verbose output
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               verbose=True)

## Datetime

In [51]:
import datetime

current_date = datetime.datetime.now()

formatted_date = current_date.strftime('%Y-%m-%d')

# Create a query string that includes the formatted date.
query = f"Who is Divyansh Garg , a candidate master on codeforces {formatted_date}? "

## Invoking Agent

In [52]:
# Invoke the agent executor 
result = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Divyansh Garg codeforces candidate master'}`


[{'url': 'https://codeforces.com/profile/windreaper', 'content': 'Before contest Codeforces Round 975 ... Divyansh Garg, New Delhi, India. From IIT Kanpur. Contest rating: 1779 (max. candidate master, 2011) Contribution: -4; Friend of: 386 users ; Last visit: 25 hours ago; Registered: 4 years ago; Blog entries (0) ...'}, {'url': 'https://paperswithcode.com/author/divyansh-garg', 'content': 'Divyansh Garg | Papers With Code Stay informed on the latest trending ML papers with code, research developments, libraries, methods, and datasets. 1 code implementation • NeurIPS 2020 • Divyansh Garg, Yan Wang, Bharath Hariharan, Mark Campbell, Kilian Q. 1 code implementation • CVPR 2020 • Rui Qian, Divyansh Garg, Yan Wang, Yurong You, Serge Belongie, Bharath Hariharan, Mark Campbell, Kilian Q. 1 code implementation • ICLR 2020 • Yurong You, Yan Wang, Wei-L

## Printing Results

In [53]:
# Print a summary of the query on the specified date
print(f"{formatted_date}:")
print(result['output'])

2024-09-30:
Divyansh Garg is a candidate master on Codeforces. He is from New Delhi, India and is a student at IIT Kanpur. He has a Codeforces rating of 1779, which is the maximum for a candidate master. He has been a member of Codeforces for 4 years and has contributed to the platform by helping other users. He is also a researcher in the field of computer vision and has published several papers on the topic. 



In [54]:
from nltk.tokenize import sent_tokenize
import textwrap

sentences = sent_tokenize(result['output'])

wrapper = textwrap.TextWrapper(width=80)

print(f"{formatted_date}:\n")
for sentence in sentences:
    wrapped_text = wrapper.fill(sentence)
    print(f"- {wrapped_text}\n")

2024-09-30:

- Divyansh Garg is a candidate master on Codeforces.

- He is from New Delhi, India and is a student at IIT Kanpur.

- He has a Codeforces rating of 1779, which is the maximum for a candidate master.

- He has been a member of Codeforces for 4 years and has contributed to the
platform by helping other users.

- He is also a researcher in the field of computer vision and has published
several papers on the topic.

